# Cloudflare Security Demo - Testing Scripts

This notebook contains all the scripts needed for demonstrating Cloudflare security features.

**Demo Site:** https://demo.oskarcode.com/

**Usage:** Run each cell individually during your demo presentation.

## 1. DNS & Proxy Validation (Onboarding)

These commands verify that the domain is properly configured with Cloudflare.

In [ ]:
# DNS Resolution Check
# Purpose: Verify domain resolves to Cloudflare IPs
# Expected: Should return Cloudflare IP addresses (104.x.x.x, 172.x.x.x, etc.)
!dig +short demo.oskarcode.com

In [ ]:
# HTTP Headers Check
# Purpose: Verify Cloudflare is proxying the requests
# Expected: Look for 'server: cloudflare' and 'cf-ray' headers
# What it means: cf-ray = Cloudflare request ID, server = Cloudflare edge server
!curl -I https://demo.oskarcode.com/

In [ ]:
# Whois IP Lookup
# Purpose: Verify the resolved IP belongs to Cloudflare
# Expected: Should show Cloudflare, Inc. as the organization
# What it means: Confirms traffic goes through Cloudflare's network
import subprocess
import socket

# Get IP address
domain_ip = socket.gethostbyname('demo.oskarcode.com')
print(f"Domain IP: {domain_ip}")

# Run whois on the IP
result = subprocess.run(['whois', domain_ip], capture_output=True, text=True)
for line in result.stdout.split('\n'):
    if 'org' in line.lower() or 'organization' in line.lower():
        print(line.strip())

## 2. SQL Injection Testing

Testing WAF (Web Application Firewall) protection against SQL injection attacks.

In [ ]:
# Basic SQL Injection Test
# Purpose: Test if WAF blocks SQL injection attempts
# Expected WITHOUT Cloudflare: Returns products from database
# Expected WITH Cloudflare WAF: HTTP 403 Forbidden or challenge page
# What it means: Cloudflare WAF detected and blocked malicious SQL payload
!curl -s "https://demo.oskarcode.com/search/?q=test' OR '1'='1' --" | head -10

In [ ]:
# Advanced SQL Injection Test
# Purpose: Test WAF against more sophisticated SQL injection
# Expected: Should be blocked by Cloudflare WAF
# What it means: Advanced threat protection is working
!curl -s "https://demo.oskarcode.com/search/?q=test' UNION SELECT null,username,password FROM users--" | head -10

In [ ]:
# Check Response Headers for SQL Injection
# Purpose: See HTTP status and Cloudflare security headers
# Expected: HTTP 403 with cf-mitigated header indicating block reason
# What it means: Shows exactly how Cloudflare blocked the threat
!curl -I "https://demo.oskarcode.com/search/?q=test' OR '1'='1' --"

## 3. Credential Exposure Testing

Testing protection against exposed sensitive files and credentials.

In [ ]:
# Environment File Exposure Test
# Purpose: Test if sensitive .env files are accessible
# Expected WITHOUT Cloudflare: Shows database credentials and API keys
# Expected WITH Cloudflare: HTTP 403 or blocked by access rules
# What it means: Cloudflare Access Rules protect sensitive configuration files
!curl -s https://demo.oskarcode.com/.env.backup

In [ ]:
# Git Secrets Exposure Test
# Purpose: Test if git repository files with secrets are accessible
# Expected: Should be blocked by Cloudflare Access Rules
# What it means: Protects against common credential leaking vectors
!curl -s https://demo.oskarcode.com/.git/secrets.txt

In [ ]:
# Database Configuration Test
# Purpose: Test if database config files are exposed
# Expected: Should return HTTP 403 or 404
# What it means: Critical infrastructure details are protected
!curl -I https://demo.oskarcode.com/config/database.yml

## 4. Bot Detection and Management

Testing Cloudflare's ability to detect and block malicious bots.

In [ ]:
# Bad Bot User Agent Test
# Purpose: Test if Cloudflare Bot Management detects malicious bots
# Expected WITHOUT Bot Management: Normal page content
# Expected WITH Bot Management: Challenge page or HTTP 403
# What it means: Suspicious user agents are flagged and challenged
!curl -I "https://demo.oskarcode.com/robots-welcome" -A "BadBot/1.0"

In [ ]:
# Scraping Bot Test
# Purpose: Test bot detection on content scraping attempts
# Expected: Should trigger bot challenge or block
# What it means: Protects intellectual property and content
!curl -s "https://demo.oskarcode.com/robots-welcome" -A "ScrapingBot/2.1 (+http://example.com/bot)" | head -10

In [ ]:
# Headless Browser Detection
# Purpose: Test detection of automated browsers used for scraping
# Expected: Challenge or block headless browser attempts
# What it means: Advanced bot detection beyond just user-agent strings
!curl -I "https://demo.oskarcode.com/" -A "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) HeadlessChrome/91.0.4472.124 Safari/537.36"

In [ ]:
# Good Bot vs Bad Bot Comparison
# Purpose: Show how legitimate bots are allowed while bad ones are blocked
# Expected: Google bot should be allowed, BadBot should be challenged
# What it means: Cloudflare intelligently distinguishes between bot types
print("=== GOOD BOT (Google) ===")
!curl -I "https://demo.oskarcode.com/" -A "Googlebot/2.1 (+http://www.google.com/bot.html)"
print("\n=== BAD BOT ===")
!curl -I "https://demo.oskarcode.com/" -A "BadBot/1.0"

## 5. Rate Limiting Testing

Testing Cloudflare Rate Limiting to prevent abuse and DDoS attacks.

In [ ]:
# API Rate Limiting Test
# Purpose: Test if rate limiting prevents API abuse
# Expected WITHOUT Rate Limiting: All requests succeed (HTTP 200)
# Expected WITH Rate Limiting: Later requests blocked (HTTP 429)
# What it means: Prevents API abuse and ensures fair usage
print("Testing API rate limiting with 10 rapid requests...")
for i in range(1, 11):
    result = !curl -s -o /dev/null -w "Request {i}: %{{http_code}}\n" https://demo.oskarcode.com/api/products/
    print(result[0])

In [ ]:
# Login Brute Force Protection Test
# Purpose: Test rate limiting on login attempts to prevent brute force
# Expected: After several attempts, should get rate limited
# What it means: Protects user accounts from credential stuffing attacks
print("Testing login rate limiting...")
for i in range(1, 6):
    result = !curl -s -X POST -d "username=admin&password=wrong" -w "Login attempt {i}: %{{http_code}}\n" -o /dev/null https://demo.oskarcode.com/api/login-test/
    print(result[0])

In [ ]:
# Page Access Rate Limiting
# Purpose: Test rate limiting on page access to prevent scraping
# Expected: Should start blocking after threshold is reached
# What it means: Prevents automated content scraping and site abuse
print("Testing page access rate limiting...")
for i in range(1, 8):
    result = !curl -s -I https://demo.oskarcode.com/ -w "Page request {i}: %{{http_code}}\n" -o /dev/null
    print(result[0])

## 6. DDoS Protection Testing

Testing Cloudflare's DDoS protection capabilities.

In [ ]:
# Simulated Traffic Spike Test
# Purpose: Simulate sudden traffic increase to test DDoS protection
# Expected: Cloudflare should handle the load gracefully
# What it means: Site remains available during traffic spikes
import threading
import subprocess
import time

def make_request(i):
    result = subprocess.run(['curl', '-s', '-o', '/dev/null', '-w', f'Request {i}: %{{http_code}}', 'https://demo.oskarcode.com/'], capture_output=True, text=True)
    return result.stdout

print("Simulating traffic spike with 20 concurrent requests...")
threads = []
results = []

for i in range(1, 21):
    thread = threading.Thread(target=lambda x=i: results.append(make_request(x)))
    threads.append(thread)
    thread.start()

for thread in threads:
    thread.join()

for result in results:
    print(result)

In [ ]:
# Large Request Test
# Purpose: Test protection against large HTTP requests
# Expected: Should be handled or blocked appropriately
# What it means: Prevents resource exhaustion attacks
large_data = "x" * 10000  # 10KB of data
!curl -s -X POST -d "{large_data}" -w "Large request: %{{http_code}}\n" -o /dev/null https://demo.oskarcode.com/contact/

## 7. Security Headers Validation

Checking that Cloudflare adds important security headers.

In [ ]:
# Security Headers Check
# Purpose: Verify Cloudflare adds security headers
# Expected: Should see headers like X-Content-Type-Options, X-Frame-Options, etc.
# What it means: Additional layers of security protection
!curl -I https://demo.oskarcode.com/ | grep -E "(x-|cf-|strict-|content-security)"

In [ ]:
# SSL/TLS Configuration Check
# Purpose: Verify SSL configuration and Cloudflare certificates
# Expected: Should show Cloudflare SSL certificate details
# What it means: Traffic is encrypted and authenticated
!echo | openssl s_client -connect demo.oskarcode.com:443 -servername demo.oskarcode.com 2>/dev/null | openssl x509 -noout -issuer -subject -dates

## 8. Performance Testing

Testing the performance improvements provided by Cloudflare's CDN.

In [ ]:
# Page Load Time Test
# Purpose: Measure page load performance with Cloudflare CDN
# Expected: Fast load times due to edge caching
# What it means: Better user experience and SEO rankings
!curl -s -w "Page Load Times:\nDNS lookup: %{time_namelookup}s\nConnect: %{time_connect}s\nSSL handshake: %{time_appconnect}s\nTime to first byte: %{time_starttransfer}s\nTotal time: %{time_total}s\n" -o /dev/null https://demo.oskarcode.com/

In [ ]:
# Cache Status Check
# Purpose: Verify content is being cached by Cloudflare
# Expected: Should see 'cf-cache-status: HIT' after first request
# What it means: Static content served from edge, reducing origin load
print("First request (likely MISS):")
!curl -I https://demo.oskarcode.com/ | grep -i cache
print("\nSecond request (should be HIT):")
!curl -I https://demo.oskarcode.com/ | grep -i cache

## 9. Geographic Testing

Testing Cloudflare's global network and geo-based features.

In [ ]:
# Cloudflare Ray ID and Data Center
# Purpose: See which Cloudflare data center is serving the request
# Expected: cf-ray header shows location code (last 3 characters)
# What it means: Traffic routed to nearest Cloudflare edge location
!curl -I https://demo.oskarcode.com/ | grep cf-ray

In [ ]:
# Country Detection Test
# Purpose: Test if Cloudflare can detect visitor country (if configured)
# Expected: May see geo-related headers if configured
# What it means: Enables geo-based rules and content customization
!curl -I https://demo.oskarcode.com/ | grep -i "geo\|country\|location"

## 10. Workers Protection Testing

Testing custom Cloudflare Workers security logic.

In [ ]:
# Admin Portal Protection Test
# Purpose: Test custom Workers script protecting admin areas
# Expected: Should show progressive protection (warn -> block -> redirect)
# What it means: Custom security logic at the edge
print("First admin portal access (should warn):")
!curl -I https://demo.oskarcode.com/admin-portal/

In [ ]:
# Multiple Admin Access Test
# Purpose: Test Workers progressive protection logic
# Expected: After 3 attempts, should redirect to demo.oskarcode.com
# What it means: Smart threat response escalation
for i in range(1, 5):
    print(f"\nAdmin attempt {i}:")
    result = !curl -I https://demo.oskarcode.com/admin-portal/ 2>/dev/null | grep -E "(HTTP|Location)"
    for line in result:
        print(line)

## 11. Analytics and Reporting

Commands to check security events and analytics.

In [ ]:
# Security Events Summary
# Purpose: Show what security events would be logged
# Expected: Educational summary of what gets logged
# What it means: Comprehensive visibility into threats and attacks
print("""
=== CLOUDFLARE SECURITY ANALYTICS ===

The following events are automatically logged in Cloudflare Analytics:

🛡️  WAF Events:
   - SQL injection attempts blocked
   - XSS attempts blocked  
   - Malicious payload detection

🤖 Bot Management:
   - Malicious bot requests challenged/blocked
   - Legitimate bot traffic allowed
   - Bot score analysis

⚡ Rate Limiting:
   - Requests blocked due to rate limits
   - Sources of excessive traffic
   - API abuse prevention

🌍 Traffic Analytics:
   - Geographic distribution of threats
   - Top attack vectors
   - Response time improvements

🔒 Access Control:
   - Blocked access to sensitive files
   - IP-based blocking events
   - Custom Workers actions

View detailed analytics at: https://dash.cloudflare.com/
""")

## 12. Demo Summary Commands

Quick summary commands to show overall protection status.

In [ ]:
# Complete Security Validation
# Purpose: Quick overview of all security features active
# Expected: Shows Cloudflare protection across all vectors
# What it means: Comprehensive security posture

print("=== CLOUDFLARE SECURITY STATUS ===")
print("\n1. DNS & Proxy Status:")
!dig +short demo.oskarcode.com | head -2

print("\n2. Cloudflare Headers:")
!curl -I https://demo.oskarcode.com/ 2>/dev/null | grep -E "(server: cloudflare|cf-ray)"

print("\n3. WAF Protection (SQL Injection Test):")
!curl -I "https://demo.oskarcode.com/search/?q=test' OR '1'='1'" 2>/dev/null | grep "HTTP"

print("\n4. Bot Management (Bad Bot Test):")
!curl -I "https://demo.oskarcode.com/robots-welcome" -A "BadBot/1.0" 2>/dev/null | grep "HTTP"

print("\n5. SSL/TLS Certificate:")
!echo | openssl s_client -connect demo.oskarcode.com:443 -servername demo.oskarcode.com 2>/dev/null | openssl x509 -noout -issuer | grep -o "Cloudflare"

print("\n✅ All Cloudflare security features are active and protecting the site!")

## Notes for Demo Presentation

### Key Points to Highlight:

1. **Before Cloudflare:** Show what happens without protection
2. **After Cloudflare:** Demonstrate how threats are blocked
3. **Performance:** Highlight speed improvements
4. **Analytics:** Explain visibility and reporting
5. **Scalability:** Show how it handles traffic spikes

### Expected Response Codes:
- **200:** Normal, allowed request
- **403:** Blocked by security rules
- **429:** Rate limited
- **503:** Challenge page ("Just a moment...")

### Key Headers to Point Out:
- `server: cloudflare` - Confirms traffic through Cloudflare
- `cf-ray` - Unique request ID for troubleshooting
- `cf-cache-status` - Shows caching effectiveness
- `cf-mitigated` - Shows what security action was taken

### Demo Flow Suggestion:
1. Start with DNS/Proxy validation
2. Show vulnerability tests (SQL injection, bots, etc.)
3. Demonstrate rate limiting and DDoS protection
4. Highlight performance improvements
5. End with analytics overview

**Remember:** Run each cell individually during your demo for maximum impact!